In [1]:
from mmdeploy_runtime import Detector,PoseDetector
import cv2
import numpy as np
import time
from copy import deepcopy 

In [2]:
det_model = Detector('rtm-pose/rtmdet-nano/','cuda')
pose_model = PoseDetector('rtm-pose/rtmpose-m/','cuda')

In [3]:
img = cv2.imread('mmdeploy/demo/resources/human-pose.jpg')


In [4]:
result = det_model(img)

In [5]:
bboxes, labels, _ = result

In [8]:
bboxes

array([[7.03422666e-01, 6.54465139e-01, 2.16919708e+02, 3.30006409e+02,
        8.47006977e-01],
       [6.32764320e+01, 8.53223354e-02, 1.10888824e+02, 7.71810379e+01,
        1.94046050e-01],
       [1.38146271e+02, 3.14815826e+02, 2.17000000e+02, 3.45000000e+02,
        1.10033423e-01],
       [1.22944153e+02, 5.92403946e+01, 2.15467850e+02, 3.31181549e+02,
        9.80338156e-02],
       [1.44086365e+02, 3.25847107e+02, 2.11675110e+02, 3.45000000e+02,
        9.79310274e-02],
       [1.46952393e+02, 2.95056366e+02, 1.59545029e+02, 3.22895538e+02,
        9.57999825e-02],
       [1.29044434e+02, 2.81792725e+02, 1.45641663e+02, 3.23232452e+02,
        9.53992903e-02],
       [1.46979340e+02, 2.85716339e+02, 1.64834717e+02, 3.23786163e+02,
        8.24018717e-02],
       [7.44832535e+01, 1.84639908e+02, 1.46730881e+02, 3.21973999e+02,
        7.80263841e-02],
       [2.00250238e-01, 1.75701484e-01, 1.17353691e+02, 3.03171570e+02,
        7.67248273e-02],
       [7.95193024e+01, 1.6855

In [9]:
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.05)
bboxes1 = bboxes[keep,:4]

In [11]:
bboxes1.shape

(24, 4)

In [ ]:
poses = pose_model(img, bboxes1)

In [ ]:
def visualize(frame, keypoints, thr=0.5, resize=None):
    skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
                (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
                (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
    palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
               (255, 153, 255), (153, 204, 255), (255, 102, 255),
               (255, 51, 255), (102, 178, 255),
               (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
               (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
               (0, 0, 255), (255, 0, 0), (255, 255, 255)]
    link_color = [
        0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
    ]
    point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]
    img = deepcopy(frame)
    scores = keypoints[..., 2]
    keypoints = (keypoints[..., :2]).astype(int)
    if resize:
        scale = resize / max(frame.shape[0], frame.shape[1])

       
        keypoints = (keypoints[..., :2] * scale).astype(int)

        img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    for kpts, score in zip(keypoints, scores):
        show = [0] * len(kpts)
        for (u, v), color in zip(skeleton, link_color):
            if score[u] > thr and score[v] > thr:
                cv2.line(img, kpts[u], tuple(kpts[v]), palette[color], 1,
                         cv2.LINE_AA)
                show[u] = show[v] = 1
        for kpt, show, color in zip(kpts, show, point_color):
            if show:
                cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
    return img

img_o = visualize(img,poses,thr=0.5)
cv2.imshow('',img_o)
cv2.waitKey(-1)